# Strength Level CSV to Day One CSV

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tkinter as tk
import datetime
from tkinter import filedialog
import ipywidgets as widgets

### Import Strength Level CSV

In [ ]:
root = tk.Tk()
root.withdraw()
root.call("wm", "attributes", ".", "-topmost", True)
file_path = filedialog.askopenfilename(
    title="Select Strength Level CSV", initialdir=os.getcwd()
)
print("Selected file:")
print(file_path)

### Import dataframe

In [ ]:
df_SL = pd.read_csv(file_path, sep=",", header=0, index_col=None)

### Summary statistics

In [ ]:
n_workouts = len(np.unique(df_SL["Date Lifted"]))
print("Total number of workouts: {}".format(n_workouts))

exercises = np.unique(df_SL["Exercise"])
n_exercises = len(exercises)
print("Number of different exercises performed: {}".format(n_exercises))

df_SL["Datetime"] = [
    datetime.datetime.strptime(date, "%Y-%m-%d") for date in df_SL["Date Lifted"]
]
date_range = (df_SL["Datetime"].iloc[0] - df_SL["Datetime"].iloc[-1]).days
train_frequency = n_workouts / (date_range / 7)
print("Training frequency: {} workouts per week".format(np.round(train_frequency, 1)))
print(" ")

exercise_count = np.array(
    [
        len(np.unique(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]))
        for exercise in exercises
    ]
)
exercise_sortidx = np.flip(np.argsort(exercise_count))
exercises_sorted = exercises[exercise_sortidx]
exercise_count_sorted = exercise_count[exercise_sortidx]

exercise_sets = np.array(
    [len(df_SL["Date Lifted"][df_SL["Exercise"] == exercise]) for exercise in exercises]
)
exercise_sets_sorted = exercise_sets[exercise_sortidx]

exercise_volume = np.array(
    [
        np.sum(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_volume_sorted = exercise_volume[exercise_sortidx]

exercise_record = np.array(
    [
        np.max(df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_sorted = exercise_record[exercise_sortidx]

exercise_record_bw = (
    exercise_record
    / np.array(
        [
            (
                df_SL["Bodyweight (kg)"][
                    (df_SL["Exercise"] == exercise)
                    & (df_SL["Weight (kg)"] == exercise_record[i])
                ]
            ).min()
            for i, exercise in enumerate(exercises)
        ]
    )
    * 100
)
exercise_record_bw_sorted = exercise_record_bw[exercise_sortidx]

exercise_record_perc = np.array(
    [
        np.max(df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])
        for exercise in exercises
    ]
)
exercise_record_perc_sorted = exercise_record_perc[exercise_sortidx]

n_top = 6
print("Top {} most performed exercises: ".format(n_top))
for i, exercise in enumerate(exercises_sorted[:n_top]):
    print(
        "     {}: \n        {} times, \n        {} kg max weight lifted ({}% bodyweight), \n        {} percentile best set, \n        {} sets, \n        {} kg total volume".format(
            exercise,
            exercise_count_sorted[i],
            exercise_record_sorted[i],
            np.round(exercise_record_bw_sorted[i], 1),
            exercise_record_perc_sorted[i],
            exercise_sets_sorted[i],
            np.round(exercise_volume_sorted[i], 1),
        )
    )

### Progress plot

In [ ]:
fig, axis = plt.subplots(1,2, figsize=(12,5))

for i, exercise in enumerate(exercises_sorted[:n_top]):
    axis[0].plot(df_SL["Datetime"][df_SL["Exercise"] == exercise], df_SL["Weight (kg)"][df_SL["Exercise"] == exercise])
    axis[1].plot(df_SL["Datetime"][df_SL["Exercise"] == exercise], df_SL["Percentile (%)"][df_SL["Exercise"] == exercise])

axis[0].set_xlabel("Date")
axis[0].set_ylabel("Weight (kg)")
axis[0].set_title("Weight")
axis[0].grid()
axis[1].set_xlabel("Date")
axis[1].set_ylabel("Percentile (%)")
axis[1].set_title("Percentile")
axis[1].grid()
fig.legend(exercises_sorted[:n_top], loc="center left", bbox_to_anchor=(0.9, 0.5))

### Select start date

In [ ]:
dates_unique = np.unique(df_SL["Date Lifted"])

date_select = widgets.Select(
    options=dates_unique,
    value=dates_unique[0],
    description="Start date:",
    style={"description_width": "initial"},
)

confirm_button = widgets.Button(description="Confirm", disabled=False)
display(date_select, confirm_button)


def confirm_button_func(a):
    global start_date
    start_date = date_select.value

    print("Selected start date: " + start_date)
    return start_date


confirm_button.on_click(confirm_button_func)

### Rearrange dataframe

In [ ]:
start_date_idx = np.where(df_SL["Date Lifted"] == start_date)[0][-1]
df_SL_select = df_SL.iloc[: start_date_idx + 1]

In [ ]:
df_SL_select["Date Lifted"] = [
    (
        datetime.datetime.strptime(item, "%Y-%m-%d") + datetime.timedelta(hours=12)
    ).isoformat()
    + ".000+02:00"
    for item in df_SL_select["Date Lifted"]
]
date_DO = np.unique(df_SL_select["Date Lifted"])
text_DO = []

for i, date in enumerate(date_DO):
    df_date = df_SL_select[df_SL_select["Date Lifted"] == date]
    _, idx = np.unique(df_date["Exercise"], return_index=True)
    sorted_idx = np.sort(idx)
    exercises = df_date["Exercise"].iloc[sorted_idx]

    date_str = ""

    for j, exercise in enumerate(exercises):
        weights = np.unique(df_date["Weight (kg)"][df_date["Exercise"] == exercise])
        exercise_str = exercise + " "
        for k, weight in enumerate(weights):
            if k == 0:
                exercise_str = (
                    exercise_str + str(np.round(weight, decimals=1)) + " kg: "
                )
            else:
                exercise_str = (
                    exercise_str + " | " + str(np.round(weight, decimals=1)) + " kg: "
                )
            reps = df_date["Reps"][
                (df_date["Exercise"] == exercise) & (df_date["Weight (kg)"] == weight)
            ]
            _, idx = np.unique(reps, return_index=True)
            sorted_idx = np.sort(idx)
            unique_reps = reps.iloc[sorted_idx]
            for l, unique_rep in enumerate(unique_reps):
                n_sets = np.sum(reps == unique_rep)
                if l == len(unique_reps) - 1:
                    exercise_str = exercise_str + str(n_sets) + "x" + str(unique_rep)
                else:
                    exercise_str = (
                        exercise_str + str(n_sets) + "x" + str(unique_rep) + ", "
                    )

        if j == 0:
            date_str = date_str + exercise_str
        else:
            date_str = date_str + "\n" + exercise_str

    date_str = "Workout " + date[:10] + "\n" + date_str
    text_DO.append(date_str)

df_DO = pd.DataFrame({"date": date_DO, "text": text_DO})

### Export to CSV

In [ ]:
csv_name = "SLtoDO_" + start_date + "_" + str(date_DO[-1][:10]) + ".csv"
df_DO.to_csv("Output/" + csv_name, sep=",", header=True, index=False)